In [13]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
import sys

from nltk.corpus import PlaintextCorpusReader
import os
import sys
wordlists = PlaintextCorpusReader("Nepali_Corpus", '.*txt')
data = wordlists.fileids()[:10]
text = []
invalid = 0
for i in data:
    with open(os.path.join("Nepali_Corpus",i),encoding='utf8') as file:
        try:
            text.append(file.read())
        except UnicodeDecodeError :
            print(i)
            invalid+=1
print(invalid)
raw_text = "".join(text)
raw_text = raw_text.replace("\n","")

0


In [14]:

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model


[' ', '(', ')', ',', '.', '?', '\xa0', '¥', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '।', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '–', '‘', '’']
Total Characters:  13424
Total Vocab:  78
Total Patterns:  13224


In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
13224/13224 [==============================] - 397s 30ms/step - loss: 3.5163

Epoch 00001: loss improved from inf to 3.51626, saving model to weights-improvement-01-3.5163-bigger.hdf5
Epoch 2/50
13224/13224 [==============================] - 420s 32ms/step - loss: 3.4581

Epoch 00002: loss improved from 3.51626 to 3.45811, saving model to weights-improvement-02-3.4581-bigger.hdf5
Epoch 3/50
13224/13224 [==============================] - 432s 33ms/step - loss: 3.4238

Epoch 00003: loss improved from 3.45811 to 3.42379, saving model to weights-improvement-03-3.4238-bigger.hdf5
Epoch 4/50
13224/13224 [==============================] - 410s 31ms/step - loss: 3.3498

Epoch 00004: loss improved from 3.42379 to 3.34984, saving model to weights-improvement-04-3.3498-bigger.hdf5
Epoch 5/50
13224/13224 [==============================] - 570s 43ms/step - loss: 3.2215

Epoch 00005: loss improved from 3.34984 to 3.22149, saving model to weights-improvement-05-3.2215-bigger.hdf5
Epoch 6/5

13224/13224 [==============================] - 389s 29ms/step - loss: 0.9385

Epoch 00042: loss improved from 0.97310 to 0.93846, saving model to weights-improvement-42-0.9385-bigger.hdf5
Epoch 43/50
13224/13224 [==============================] - 391s 30ms/step - loss: 0.8801

Epoch 00043: loss improved from 0.93846 to 0.88010, saving model to weights-improvement-43-0.8801-bigger.hdf5
Epoch 44/50
13224/13224 [==============================] - 387s 29ms/step - loss: 0.8378

Epoch 00044: loss improved from 0.88010 to 0.83776, saving model to weights-improvement-44-0.8378-bigger.hdf5
Epoch 45/50
13224/13224 [==============================] - 384s 29ms/step - loss: 0.8178

Epoch 00045: loss improved from 0.83776 to 0.81778, saving model to weights-improvement-45-0.8178-bigger.hdf5
Epoch 46/50
13224/13224 [==============================] - 390s 30ms/step - loss: 0.7835

Epoch 00046: loss improved from 0.81778 to 0.78352, saving model to weights-improvement-46-0.7835-bigger.hdf5
Epoch 47/50


In [18]:
model.save("2-layer-200-weights.h5")

In [25]:
model = keras.models.load_model("2-layer-200-weights.h5")

In [35]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print (''.join([int_to_char[value] for value in pattern]), "\n")
print("".join([int_to_char[value] for value in pattern]),end="")
# generate characters
f = open("sent_dump","w")
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    print(result,sep='',end='')
    f.write("".join([int_to_char[int(j*n_vocab)] 
            for i in x[0]
               for j in i
           ])+"-->"+result+"\n")
    pattern.append(index)
    
    pattern = pattern[1:len(pattern)]
print ("\nDone.")
f.close()

Seed:
ी मागेको पनि खुलासा गरिन्। फिल्म गुलामको अभिनेत्रीमा छानिएकी रानीको आवाज मोटो भएको र त्यो फिल्ममा नसुहाउने भन्दै प्रमुख अभिनेता आमिर खान, निर्देशक विक्रम भट्ट र निर्माता मुकेश भट्टले उनको भुमिकाका लाग 

ी मागेको पनि खुलासा गरिन्। फिल्म गुलामको अभिनेत्रीमा छानिएकी रानीको आवाज मोटो भएको र त्यो फिल्ममा नसुहाउने भन्दै प्रमुख अभिनेता आमिर खान, निर्देशक विक्रम भट्ट र निर्माता मुकेश भट्टले उनको भुमिकाका लागि आवाज डबिङ्ग हुने भएको छुनासो पनि सुन्नुपर्याक चन्दले आफूले विगतमा प्रुष्यिपक सहेकरले जाणीलाई प्राण गरेका थिए । फाटक घरको उदघाट च्रदाशन गर्ने सम्झिला माम्दै  दाज  थार्मेत  भ्रानिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रिनिन  प्रि